## Notebook config

In [ ]:
DATA_CSV = "data.csv"
STATIONS_CSV = "stations.csv"
SHAPEFILE = "oslo-shapefile/roads.shp"

## Read data

In [ ]:
from bike_analysis import *
df = read_augmented_availability_data(DATA_CSV, STATIONS_CSV)
stations = read_station_data(STATIONS_CSV)

## Plotting functions

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt

def plot_station_usage(df, stations):
    mpl.style.use("seaborn")
    for station_id in stations:
        sub_df = df[df.station_id == station_id]
        sub_df['real_capacity'] = sub_df.num_bikes_available + sub_df.num_docks_available
        sub_df[['num_bikes_available', 'real_capacity']].plot()
        top_row = sub_df.iloc[0]
        plt.ylim(0, top_row.station_capacity)
        plt.title(top_row.station_name)
        plt.show();

## Bike availability over time and space

In [ ]:
xlim = (10.65, 10.82)
ylim = (59.89, 59.96)
gdf = get_road_gdf(SHAPEFILE, xlim, ylim)

In [ ]:
import numpy as np
from pandas import Timedelta

mpl.style.use("grayscale")

max_snaps = np.inf
snap_i = 0
prev_ts = None
for snap in snapshots(df):
    if prev_ts is None:
        prev_ts = snap.index[0]
    else:
        timediff = snap.index[0] - prev_ts
        prev_ts = snap.index[0]
    plt.figure(figsize=(15, 10))
    ax = plt.gca()
    snap.plot.scatter(
        'station_lon',
        'station_lat',
        s=snap.station_capacity*8,
        c=100*snap.num_bikes_available/snap.station_capacity,
        colorbar=True,
        colormap='RdBu',
        grid=False,
        alpha=0.75,
        ax=ax
    )
    gdf.plot(alpha=0.25, ax=ax, color="black")
    plt.axis('auto')
    plt.ylim(ylim)
    plt.xlim(xlim)
    plt.title(snap.index[0])
    plt.savefig("snap_{:05d}".format(snap_i))
    plt.close()
    snap_i += 1
    if snap_i >= max_snaps:
        break